In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os

In [2]:
os.chdir("C:\\Users\\gunsu\Documents\\DL_Linkedin\\Ex_Files_TensorFlow\\Exercise Files\\04")

In [3]:
# Load training data set from CSV file
training_data_df = pd.read_csv("sales_data_training.csv", dtype=float)

In [4]:
# Pull out columns for X (data to train with) and Y (value to predict)
X_training = training_data_df.drop('total_earnings', axis=1).values
Y_training = training_data_df[['total_earnings']].values

In [5]:
# Load testing data set from CSV file
test_data_df = pd.read_csv("sales_data_test.csv", dtype=float)

In [6]:
# Pull out columns for X (data to train with) and Y (value to predict)
X_testing = test_data_df.drop('total_earnings', axis=1).values
Y_testing = test_data_df[['total_earnings']].values

In [7]:
# All data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well. Create scalers for the inputs and outputs.
X_scaler = MinMaxScaler(feature_range=(0, 1))
Y_scaler = MinMaxScaler(feature_range=(0, 1))

In [8]:
# Scale both the training inputs and outputs
X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)

In [10]:
#training and test data are scaled with the same scaler.
X_scaled_testing = X_scaler.transform(X_testing)
Y_scaled_testing = Y_scaler.transform(Y_testing)

In [11]:
#define model parameters
learning_rate=0.001
training_epochs=100
display_step=5

#define how many inputs and outputs are in the NN
number_of_inputs=9
number_of_outputs=1

In [12]:
number_of_inputs=9
number_of_outputs=1

In [13]:
#define how many neurons we want in each layer
layer_1_nodes=50
layer_2_nodes=100
layer_3_nodes=50

#Section one:defining layers of NN itself

In [15]:
#input layer: None tells tf that tf can accept batch of any size
with tf.variable_scope('input'):
    X=tf.placeholder(tf.float32,shape=(None,number_of_inputs))

In [17]:
# Layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable(name="weights1", shape=[number_of_inputs, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

In [19]:
# Layer 2
with tf.variable_scope('layer_2', reuse=None):
    weights = tf.get_variable(name="weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

In [20]:
# Layer 3
with tf.variable_scope('layer_3'):
    weights = tf.get_variable(name="weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

In [21]:
# Output Layer
with tf.variable_scope('output'):
    weights = tf.get_variable(name="weights4", shape=[layer_3_nodes, number_of_outputs], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases4", shape=[number_of_outputs], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases


# Section Two: Define the cost function of the neural network that will measure prediction accuracy during training

In [22]:
with tf.variable_scope('cost'):
    Y=tf.placeholder(tf.float32,shape=(None,1))
    cost=tf.reduce_mean(tf.squared_difference(prediction,Y))

In [23]:
#Optimizer tat will run to opt.to train the NN
with tf.variable_scope('train'):
    optimizer=tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [24]:
#initialize a summary operation that we can run tf operations
with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost',cost)
    summary=tf.summary.merge_all()
    

In [25]:
#Initialize a session to run TensorFlow graph (operations)

with tf.Session() as session:
    #run the global vaiable initializer to initialize all variables and layers_lway
    #initialze with tf.global_variables_initializer
    session.run(tf.global_variables_initializer())
    
    #create log file writers to record training progress
    #will store training and testing log data separately
    training_writer=tf.summary.FileWriter("./logs/training",session.graph)
    testing_writer=tf.summary.FileWriter("./logs/testing",session.graph)
    
    #run the optimizer many times to train the NN
    #One epoch is one full run through the training data set
    for epoch in range(training_epochs):
        #Feed in the training data set and do one step NN training
        session.run(optimizer,feed_dict={X:X_scaled_training,Y:Y_scaled_training})
        if epoch%5==0:
            training_cost,training_summary=session.run([cost,summary],feed_dict={X:X_scaled_training,Y:Y_scaled_training})
            testing_cost,testing_summary=session.run([cost,summary],feed_dict={X:X_scaled_training,Y:Y_scaled_training})
            #write the current training status to a log file
            training_writer.add_summary(training_summary,epoch)
            testing_writer.add_summary(testing_summary,epoch)
            
            print(epoch,training_cost,testing_cost)
        #print the current training status the the screen
    print('Training is complete!')

    final_training_cost=session.run(cost,feed_dict={X:X_scaled_training,Y:Y_scaled_training})
    final_testing_cost=session.run(cost,feed_dict={X:X_scaled_training,Y:Y_scaled_training}) 
    
    print("final_training_cost: {}".format(final_training_cost))
    print("final_testing_cost: {}".format(final_testing_cost))

0 0.0430876 0.0430876
5 0.024254 0.024254
10 0.00999026 0.00999026
15 0.0102059 0.0102059
20 0.0056957 0.0056957
25 0.00444549 0.00444549
30 0.00357561 0.00357561
35 0.00226845 0.00226845
40 0.00169878 0.00169878
45 0.00130227 0.00130227
50 0.000978923 0.000978923
55 0.000779158 0.000779158
60 0.00060873 0.00060873
65 0.000474125 0.000474125
70 0.000377004 0.000377004
75 0.000305243 0.000305243
80 0.000251688 0.000251688
85 0.000212215 0.000212215
90 0.000181756 0.000181756
95 0.000158934 0.000158934
Training is complete!
final_training_cost: 0.00014327651297207922
final_testing_cost: 0.00014327651297207922
